In [15]:
import cv2 as cv
import numpy as np
import matplotlib.path as mplpath

In [16]:
def getCannyEdge(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    # cv.imwrite('./Outputs/gray.png',gray)
    # gray = cv.GaussianBlur(gray,(5,5),0)
    filter = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    filtered=cv.filter2D(gray,-1,filter)
    # cv.imwrite('./Outputs/filtered.png',filtered)

    thresh = cv.threshold(filtered, 128, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)[1]
    # cv.imwrite('./Outputs/thresh.png',thresh)

    Canny = np.zeros(thresh.shape, dtype=np.uint8)
    contours, hierarchy  = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    i = 0
    for c in contours:
        perimeter = cv.arcLength(c, True)
        storeArea = cv.approxPolyDP(c, 0.02 * perimeter, True)
        if(4 <= len(storeArea) <=6):
            [Next, Previous, First_Child, Parent] = hierarchy[0][i]
            if(First_Child != -1 and Parent != -1):
                cv.drawContours(Canny, [c], -1, (255,255,255), 2)
        i += 1
    Canny = cv.bitwise_not(Canny)

    return Canny

In [17]:
def getStoreBoundries(img):
    res_img = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    # Rcontours, hierarchy  = cv.findContours(img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    contours, hierarchy  = cv.findContours(img, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    for c in contours:
        if(4 <= len(c) <= 8):
            perimeter = cv.arcLength(c, True)
            storeArea = cv.approxPolyDP(c, 0.02 * perimeter, True)
            if(4 <= len(storeArea) <= 6 ):
                cv.drawContours(res_img, [c], -1, (255,255,255), 2)
                # print(j,len(storeArea))
                # print(storeArea)
    res_img = cv.bitwise_not(res_img)
    return res_img

In [18]:
def findstore(vertices,point):
    polygon = mplpath.Path(vertices)
    return polygon.contains_point(point)

In [19]:
# image = cv.imread('./Blueprints/i1.png')
image = cv.imread('./Blueprints/i3.jpg')
CannyEdged = getCannyEdge(image)
cv.imwrite('./Outputs/CannyEdged.png',CannyEdged)
storeImage = getStoreBoundries(CannyEdged)
cv.imwrite('./Outputs/storeImage.png',storeImage)

True